<a href="https://colab.research.google.com/github/Wiickz/MAT421/blob/main/HW10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Section 22.1: ODE Initial Value Problem Statement

Ordinary differential equations (ODEs) are differential equations with only one independent value (as an example, the function *y = f(x)*). These differential equations create simpler scenarios, but are still capable of modeling various real-world phenomena such as projectile motion. When solving ODEs, it's common that the first result is a function *g(x)* that still contains unknown variables/coefficients - known as a general solution - and in order to solve for those missing values and find a/the particular solution, a set of initial conditions is typically provided, which is just enough info to solve for the particular equation(s).

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# first order ODE y' + ycos(t) = 0
# general solution y = C*exp(-sin(t))
# with initial condition y(0) = a, we have C = a

y = lambda a, t: a * np.exp(-np.sin(t)) # to illustrate different initial conditions

t = np.linspace(0, 10, 100)
for a in np.linspace(1, 5, 5):
    plt.plot(t, y(a, t), label=f'a={int(a)}')
plt.legend()
plt.xlabel('t')
plt.ylabel('y')
plt.title('First order ODE y\' + ycos(t) = 0')
plt.show()

# Section 22.2: Reduction of Order

For first-order ODEs with less obvious solutions than the one above, it's usually helpful to employ the use of pre-written ODE solvers such as SciPy's `integrate.solve_ivp()` function:

In [ ]:
import scipy.integrate as spi
import numpy as np
import matplotlib.pyplot as plt

# first order ODE y' = ysin(y)
# initial condition y(0) = 1

def f(t, y):
    return y * np.sin(y)

t = np.linspace(0, 10, 100)
y0 = 1
sol = spi.solve_ivp(f, (0, 10), [y0], t_eval=t)
plt.plot(sol.t, sol.y[0])
plt.xlabel('t')
plt.ylabel('y')
plt.title('First order ODE y\' = ysin(y), y(0) = 1')
plt.show()

However, this function and many others are only designed to handle first-order ODEs. In order to handle higher-order ODEs, they must first be converted to first-order and then plugged in. Fortunately, a method exists in which a set of higher-order equations can be represented as a single state equation whose derivative can be taken to create a new (first-order) ODE:

In [ ]:
import scipy.integrate as spi
import numpy as np
import matplotlib.pyplot as plt

# system of two ODEs
# r' = 4r - 2s, s' = r + s
# initial condition r(0) = 4, s(0) = 2

# represent the system as a state equation S(t) = [r(t), s(t)]
# then, S(t)' = [r'(t), s'(t)] = [4r - 2s, r + s] = [ [4, -2], [1, 1] ] * S(t)
# now we can use solve_ivp to solve the system

A = np.array([[4, -2], [1, 1]])
def f(t, S):
    return A @ S

t = np.linspace(0, 5, 100)
S0 = np.array([4, 2])

sol = spi.solve_ivp(f, (0, 5), S0, t_eval=t)
plt.plot(sol.t, sol.y[0], label='r')
plt.plot(sol.t, sol.y[1], label='s')
plt.legend()
plt.xlabel('t')
plt.ylabel('r, s')
plt.title('System of two ODEs r\' = 4r - 2s, s\' = r + s, r(0) = 4, s(0) = 2')
plt.show()

## Section 22.3: The Euler Method

Suppose we have solved for this first-order differential state equation, and knowing the value of the current state, wish to evaluate a future state. This evaluation can be approximated by representing the value of the next state as the sum of the current state value plus the product of the value of the differential equation at that point and the spacing between the two points in time. This is known as Euler's explicit formula and can be used repeatedly to graph the state for all values of interest, thus yielding an approximation of the ODE's solution:

In [ ]:
import scipy.integrate as spi
import numpy as np
import matplotlib.pyplot as plt

t = np.linspace(0, 5, 100)
S = np.zeros((100, 2))
S[0] = [4, 2]
A = np.array([[4, -2], [1, 1]])
dS = np.zeros((100, 2))
dS[0] = A @ S[0]
for i in range(1, 100):
    S[i] = S[i-1] + dS[i-1] * (t[i] - t[i-1]) # euler's explicit formula
    dS[i] = A @ S[i]

sol = spi.solve_ivp(f, (0, 5), S[0], t_eval=t) # exact solution

plt.figure()

plt.plot(sol.t, sol.y[0], label='r')
plt.plot(sol.t, sol.y[1], label='s')

plt.plot(t, S[:, 0], label='r_euler')
plt.plot(t, S[:, 1], label='s_euler')

plt.legend()
plt.xlabel('t')
plt.ylabel('r, s')
plt.title('System of two ODEs r\' = 4r - 2s, s\' = r + s, r(0) = 4, s(0) = 2')
plt.show()

# decreasing h
t = np.linspace(0, 5, 1000)
S = np.zeros((1000, 2))
S[0] = [4, 2]
dS = np.zeros((1000, 2))
dS[0] = A @ S[0]
for i in range(1, 1000):
    S[i] = S[i-1] + dS[i-1] * (t[i] - t[i-1]) # euler's explicit formula
    dS[i] = A @ S[i]

sol = spi.solve_ivp(f, (0, 5), S[0], t_eval=t) # exact solution

plt.figure()

plt.plot(sol.t, sol.y[0], label='r')
plt.plot(sol.t, sol.y[1], label='s')

plt.plot(t, S[:, 0], label='r_euler')
plt.plot(t, S[:, 1], label='s_euler')

plt.legend()
plt.xlabel('t')
plt.ylabel('r, s')
plt.title('Same system, h decreased by factor of 10')
plt.show()


As demonstrated above, the Euler approximation is highly dependent on the number of data points available. Increasing the number of points (and thus decreasing the spacing) yields a much more accurate result.